#  Setup stuff: Connectivity

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS = 3
   #
DB_NAME        = "my_db"
GRAPH_NAME     = "my_graph"

print("--")


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)


#  Setup stuff: Debugging

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

#  Outputted values,
#
#  24
#  42


In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, r, m
   
   """, contextualize=True)

l_result.view()


<div> 
<img src="./01_Images/Result_01.png" alt="Drawing" style="width: 1800px;"/>
</div>


In [ ]:

l_result = my_graph.query("""

   MATCH (n) 
   WITH DISTINCT LABELS(n) AS temp, COUNT(n) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   
   """, contextualize=True)

l_result.view()


# display(print(l_result))
# 
#  Outputted values
#
#     cnt        label
#  0    2  Dist_Center
#  1    4         Item
#  2    8        Order
#  3    4       Person
#  4    2     Reviewer
#  5    4        Store



In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [r] -> (m) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp

   """, contextualize=True)

l_result.view()


# display(print(l_result))
# 
#  Outputted values,
#
#                     temp  tempCnt
#  0          COMPRISED_OF        8
#  1  FOUND_AT_DIST_CENTER        6
#  2        FOUND_AT_STORE       12
#  3                PLACED        8
#  4           REVIEWED_BY        8



#  Betweeness algorithm

In [ ]:

my_graph.run(lambda g: g.nodes.remove_property("centrality"))

print("--")

#  AttributeError: 'katana_enterprise.distributed_native.Graph' object has no attribute 'nodes'


In [ ]:

from katana.remote import analytics

analytics.betweenness_centrality(
   my_graph,
      #
   result_property_name = "centrality",
   num_sources          = 100
   )

print("--")



In [ ]:

l_result = my_graph.query(
   f"""
    
   MATCH (n)
   RETURN n, n.centrality
   ORDER BY n.centrality DESC
   LIMIT 30
    
    """, contextualize=True,
   )

l_result.view()

